In [16]:
from pathlib import Path
import pandas as pd
from pandas import DataFrame
import tidytcells as tt

In [2]:
raw_dir = Path("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/raw/rds")
preprocessed_dir = Path(
    "/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/preprocessed/rds"
)

In [23]:
paths_to_tsvs = [item for item in raw_dir.iterdir() if "tsv" in item.name]

In [24]:
def load_and_clean_decombinator_tsv(path_to_tsv: Path) -> DataFrame:
    tcrs = pd.read_csv(path_to_tsv, sep="\t")
    functional_tcrs = tcrs[tcrs.productive == "T"]
    functional_tcrs_simplified = functional_tcrs[["decombinator_id", "v_call", "junction_aa", "j_call", "duplicate_count"]]
    functional_tcrs_simplified.columns = ["decombinator_id", "TRBV", "CDR3B", "TRBJ", "duplicate_count"]
    functional_tcrs_standardized = standardize_simplified_decombinator_dataframe(functional_tcrs_simplified.copy())
    return functional_tcrs_standardized

def standardize_simplified_decombinator_dataframe(simplified_decombinator_dataframe: DataFrame) -> DataFrame:
    simplified_decombinator_dataframe.TRBV = simplified_decombinator_dataframe.TRBV.map(lambda x: tt.tcr.standardize(x, enforce_functional=True))
    simplified_decombinator_dataframe.TRBJ = simplified_decombinator_dataframe.TRBJ.map(lambda x: tt.tcr.standardize(x, enforce_functional=True))
    return simplified_decombinator_dataframe.dropna()

In [25]:
tsvs = [load_and_clean_decombinator_tsv(path_to_tsv) for path_to_tsv in paths_to_tsvs]

/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRBV21-1" for species homosapiens: gene has no functional alleles. Attempted fix: "TRBV21-1".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRBV23-1" for species homosapiens: gene has no functional alleles. Attempted fix: "TRBV23-1".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRBV1" for species homosapiens: gene has no functional alleles. Attempted fix: "TRBV1".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRBV6-7" for species homosapiens: gene has no functional alleles. Attempted fix: "TRBV6-7".
  warn(
/home/yutanagano

In [28]:
combined = pd.concat(tsvs)

In [33]:
memory_like = combined.copy()[combined.duplicate_count > 1]

In [45]:
naive_like = combined.copy()[combined.duplicate_count == 1]

In [37]:
def collapse_clones(standardized_decombinator_dataframe: DataFrame) -> DataFrame:
    standardized_decombinator_dataframe["clone_count"] = 1
    collapsed_dataframe = standardized_decombinator_dataframe.groupby(
        by=["TRBV", "CDR3B", "TRBJ"],
        as_index=False
    ).aggregate({"clone_count": "sum"})
    return collapsed_dataframe

In [38]:
combined_collapsed = collapse_clones(combined)

In [41]:
memory_like_collapsed = collapse_clones(memory_like)

In [46]:
naive_like_collapsed = collapse_clones(naive_like)

In [49]:
combined_collapsed.to_csv(preprocessed_dir/"combined.csv", index=False)
memory_like_collapsed.to_csv(preprocessed_dir/"memory_like.csv", index=False)
naive_like_collapsed.to_csv(preprocessed_dir/"naive_like.csv", index=False)